In [1]:
!pip install -q shap xgboost scipy pandas matplotlib seaborn

In [10]:
from pathlib import Path

import pandas as pd
from pandas.api.types import CategoricalDtype

import numpy as np
from scipy import stats

import xgboost
import shap

import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)

## Dataset definition

In [41]:
ORDINALS_INFO = [
    ('BsmtExposure',['NA', 'No', 'Mn', 'Av', 'Gd']),
    ('BsmtQual',['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('BsmtCond', ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('ExterQual',['NA','Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('ExterCond', ['NA','Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('FireplaceQu',['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('GarageCond',['NA','Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('GarageQual',['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('HeatingQC',['NA','Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('KitchenQual',['NA','Po', 'Fa', 'TA', 'Gd', 'Ex']),
    ('LandSlope',['Gtl', 'Mod', 'Sev']),
    ('PoolQC',['NA', 'Fa', 'TA', 'Gd', 'Ex']),
    ('OverallQual', list(range(1, 11))),
    ('OverallCond', list(range(1, 11))),
    ('MoSold', None),
    ('YrSold', None),
    ('YearBuilt', None),
    ('YearRemodAdd', None),
    ('GarageYrBlt', None),
]
ORDINALS = [feat for feat, _ in ORDINALS_INFO]

NOMINALS = [
    'Alley',
    'BldgType',
    'BsmtFinType1',
    'BsmtFinType2',
    'CentralAir',
    'Condition1',
    'Condition2',
    'Electrical',
    'Exterior1st',
    'Exterior2nd',
    'Fence',
    'Foundation',
    'Functional',
    'GarageFinish',
    'GarageType',
    'Heating',
    'HouseStyle',
    'LandContour',
    'LotConfig',
    'LotShape',
    'MSSubClass',
    'MSZoning',
    'MasVnrType',
    'MiscFeature',
    'Neighborhood',
    'PavedDrive',
    'RoofMatl',
    'RoofStyle',
    'SaleCondition',
    'SaleType',
    'Utilities'
]

UNUSEDS = [
    'Street', 
    # 'MoSold', 
    # 'YrSold', 
    # 'YearBuilt', 
    # 'YearRemodAdd', 
    # 'GarageYrBlt', 
]

NUMERICALS = [
    '1stFlrSF',
    '2ndFlrSF',
    '3SsnPorch',
    'BedroomAbvGr',
    'BsmtUnfSF',
    'BsmtFinSF1',
    'BsmtFinSF2',
    'BsmtFullBath',
    'BsmtHalfBath',
    'EnclosedPorch',
    'Fireplaces',
    'FullBath',
    'GarageArea',
    'GarageCars',
    'GrLivArea',
    'HalfBath',
    'KitchenAbvGr',
    'LotArea',
    'LotFrontage',
    'LowQualFinSF',
    'MasVnrArea',
    'MiscVal',
    'OpenPorchSF',
    'PoolArea',
    'ScreenPorch',
    'TotRmsAbvGrd',
    'TotalBsmtSF',
    'WoodDeckSF',
    'SalePrice'
]

TARGETS = ['SalePrice']

def fill_missing_values_by_data_description(df):
    df = df.copy()
    for col in ORDINALS + NOMINALS:
        if col not in df.columns:
            continue
        val = 'NA' if col != 'MasVnrType' else 'None'
        df[col] = df[col].fillna(val)
    return df

def prepare_dataset(df):
    targets = list(set(df.columns).intersection(TARGETS))

    df = (df
        .drop(columns=UNUSEDS)
        .dropna(axis=0, how='any', subset=targets)
        .pipe(fill_missing_values_by_data_description)
    )

    for col in NUMERICALS:
        if col not in df.columns:
            continue
        df[col] = df[col].astype('float')
    
    for col, categories in ORDINALS_INFO:
        if col not in df.columns:
            continue
        df[col] = df[col].astype(CategoricalDtype(categories=categories, ordered=True))
    
    for col in NOMINALS:
        if col not in df.columns:
            continue
        df[col] = df[col].astype('category')
    
    for col in targets:
        if str(df[col].dtype) == 'category':
            df[col] = df[col].cat.codes
    existing_cols = set(df.columns)
    col_order =  [col for col in NUMERICALS + ORDINALS + NOMINALS if col in existing_cols]
    x = df[col_order].drop(columns = targets)
    y = df[targets]
    return x, y

In [42]:
VAR_TYPE_MAP = {
    col: t 
    for t, cols in [('numerical', NUMERICALS), ('ordinal', ORDINALS), ('nominal', NOMINALS)] 
    for col in cols
}

In [43]:
data_dir = Path('.')

df = pd.read_csv(data_dir / 'train.csv', low_memory=False, index_col='Id')
test_df = pd.read_csv(data_dir / 'test.csv', low_memory=False, index_col='Id')
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Data quality

In [44]:
def find_cols_with_missing(df):
    missing_counts = df.isna().sum()
    return missing_counts[missing_counts > 0].index.values.tolist()

def describe(df):
    desc = df.describe()
    desc.loc['n_unique', :] = df.nunique().values
    desc.loc['n_missing', :] = df.isna().sum().values
    return desc

def show_missingness(df):
    print("Missing rates")
    print('='*16)
    print(df.isna().sum() / len(df))
    print('='*16)

    msno.heatmap(df, figsize=(4,4))
    plt.show()

    msno.matrix(df, figsize=(4,4))
    plt.show()

In [45]:
xdf, ydf = prepare_dataset(df)
test_xdf, test_ydf = prepare_dataset(test_df)

In [37]:
xdf[find_cols_with_missing(xdf)].isna().sum().sort_values(ascending=False)

LotFrontage    259
MasVnrArea       8
dtype: int64

In [47]:
xdf.select_dtypes(np.number).describe()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtUnfSF,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,EnclosedPorch,...,LotFrontage,LowQualFinSF,MasVnrArea,MiscVal,OpenPorchSF,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1201.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,1162.626712,346.992466,3.409589,2.866438,567.240411,443.639726,46.549315,0.425342,0.057534,21.954110,...,70.049958,5.844521,103.685262,43.489041,46.660274,2.758904,15.060959,6.517808,1057.429452,94.244521
std,386.587738,436.528436,29.317331,0.815778,441.866955,456.098091,161.319273,0.518911,0.238753,61.119149,...,24.284752,48.623081,181.066207,496.123024,66.256028,40.177307,55.757415,1.625393,438.705324,125.338794
min,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,882.000000,0.000000,0.000000,2.000000,223.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,59.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,795.750000,0.000000
50%,1087.000000,0.000000,0.000000,3.000000,477.500000,383.500000,0.000000,0.000000,0.000000,0.000000,...,69.000000,0.000000,0.000000,0.000000,25.000000,0.000000,0.000000,6.000000,991.500000,0.000000
75%,1391.250000,728.000000,0.000000,3.000000,808.000000,712.250000,0.000000,1.000000,0.000000,0.000000,...,80.000000,0.000000,166.000000,0.000000,68.000000,0.000000,0.000000,7.000000,1298.250000,168.000000
max,4692.000000,2065.000000,508.000000,8.000000,2336.000000,5644.000000,1474.000000,3.000000,2.000000,552.000000,...,313.000000,572.000000,1600.000000,15500.000000,547.000000,738.000000,480.000000,14.000000,6110.000000,857.000000


In [49]:
xdf.select_dtypes?

Signature: xdf.select_dtypes(include=None, exclude=None) -> 'DataFrame'
Docstring:
Return a subset of the DataFrame's columns based on the column dtypes.

Parameters
----------
include, exclude : scalar or list-like
    A selection of dtypes or strings to be included/excluded. At least
    one of these parameters must be supplied.

Returns
-------
DataFrame
    The subset of the frame including the dtypes in ``include`` and
    excluding the dtypes in ``exclude``.

Raises
------
ValueError
    * If both of ``include`` and ``exclude`` are empty
    * If ``include`` and ``exclude`` have overlapping elements
    * If any kind of string dtype is passed in.

See Also
--------
DataFrame.dtypes: Return Series with the data type of each column.

Notes
-----
* To select all *numeric* types, use ``np.number`` or ``'number'``
* To select strings you must use the ``object`` dtype, but note that
  this will return *all* object dtype columns
* See the `numpy dtype hierarchy
  <https://numpy.org/doc/

In [53]:
df[ORDINALS]

,BsmtExposure,BsmtQual,BsmtCond,ExterQual,ExterCond,FireplaceQu,GarageCond,GarageQual,HeatingQC,KitchenQual,LandSlope,PoolQC,OverallQual,OverallCond,MoSold,YrSold,YearBuilt,YearRemodAdd,GarageYrBlt
Id,,,,,,,,,,,,,,,,,,,
1,No,Gd,TA,Gd,TA,NaN,TA,TA,Ex,Gd,Gtl,NaN,7,5,2,2008,2003,2003,2003.0
2,Gd,Gd,TA,TA,TA,TA,TA,TA,Ex,TA,Gtl,NaN,6,8,5,2007,1976,1976,1976.0
3,Mn,Gd,TA,Gd,TA,TA,TA,TA,Ex,Gd,Gtl,NaN,7,5,9,2008,2001,2002,2001.0
4,No,TA,Gd,TA,TA,Gd,TA,TA,Gd,Gd,Gtl,NaN,7,5,2,2006,1915,1970,1998.0
5,Av,Gd,TA,Gd,TA,TA,TA,TA,Ex,Gd,Gtl,NaN,8,5,12,2008,2000,2000,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,No,Gd,TA,TA,TA,TA,TA,TA,Ex,TA,Gtl,NaN,6,5,8,2007,1999,2000,1999.0
1457,No,Gd,TA,TA,TA,TA,TA,TA,TA,TA,Gtl,NaN,6,6,2,2010,1978,1988,1978.0
1458,No,TA,Gd,Ex,Gd,Gd,TA,TA,Ex,Gd,Gtl,NaN,7,9,5,2010,1941,2006,1941.0


## Training

In [13]:
def get_numerical_cols(df):
    return df.select_dtypes('number').columns.tolist()

def get_ordinal_cols(df):
    return [col for col in df.select_dtypes('category').columns if df[col].dtypes.ordered]

def get_nominal_cols(df):
    return [col for col in df.select_dtypes('category').columns if not df[col].dtypes.ordered]

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.decomposition import PCA

def make_preprocessor(x_train: pd.DataFrame):
    numerical_cols = get_numerical_cols(x_train)

    skewness = x_train[numerical_cols].skew()
    sym_numerical_cols = sorted(skewness[skewness.abs() <= 0.5].index.tolist())
    sym_transformer  = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
        ('scaler', StandardScaler()),
    ])

    skw_numerical_cols = sorted(list(set(numerical_cols).difference(sym_numerical_cols)))
    skw_transformer  = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
        ('scaler', MinMaxScaler()),
    ])

    ordinal_cols = sorted(get_ordinal_cols(x_train))
    ordinal_category_list = [dt.categories.tolist() for dt in x_train[ordinal_cols].dtypes]
    ordinal_transformer = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=ordinal_category_list)),
    ])

    nominal_cols = sorted(get_nominal_cols(x_train))
    nominal_transformer = Pipeline([
        ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
    ])

    preprocessor = ColumnTransformer(
        [
            ('numerical_sym', sym_transformer, sym_numerical_cols),
            ('numerical_skw', skw_transformer, skw_numerical_cols),
            ('ordinal', ordinal_transformer, ordinal_cols),
            ('nominal', nominal_transformer, nominal_cols),
        ],
        remainder='drop'
    ).fit(x_train)

    nominal_enc_cols = preprocessor.transformers_[3][1].named_steps['encoder'].get_feature_names_out(nominal_cols).tolist()
    preprocessor.feature_names_out_ = sym_numerical_cols + skw_numerical_cols + ordinal_cols + nominal_enc_cols
    
    return preprocessor 

In [9]:
x, y = prepare_dataset(df)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=7)
# x_test, _ = prepare_dataset(test_df, numericals=numericals, categs=categs)


In [ ]:
pp = make_preprocessor(x_train)
process = lambda x: pd.DataFrame(pp.transform(x), columns=pp.feature_names_out_)

In [ ]:
x_train_proc = process(x_train)
x_val_proc = process(x_val)

model = xgboost.XGBRegressor().fit(x_train_proc, y_train.values.ravel())

print(model.score(x_train_proc, y_train.values.ravel()))
print(model.score(x_val_proc, y_val.values.ravel()))

y_train_pred = model.predict(x_train_proc)

In [ ]:
explainer = shap.Explainer(model, output_names=TARGETS)
shap_values = explainer(x_train_proc)

In [ ]:
idx = 0
print(f'Prediction: {y_train_pred[idx]:.2f}')
shap.plots.waterfall(shap_values[idx],)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.summary_plot(shap_values, x_train_proc)


In [ ]:
shap.plots.scatter(shap_values[:,"OverallQual"])


In [ ]:
plt.bar(range(shap_values.values.shape[1]), shap_values.values.sum(axis=0))